In [23]:
import cv2
import dlib
import numpy as np
from scipy.spatial import distance as dist


from io import BytesIO
from IPython.display import clear_output, Image, display
from PIL import Image as Img

def padronizar_imagem(imagem):
    imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)
    imagem = cv2.resize(imagem, (800,600))
    return imagem

def exibir_video(frame):
    img = Img.fromarray(frame, "RGB")
    buffer = BytesIO()
    img.save(buffer, format="JPEG")
    display(Image(data=buffer.getvalue()))
    clear_output(wait=True)


In [12]:
captura_video = cv2.VideoCapture(0)

try:
    while(True):
        captura_ok, frame = captura_video.read()

        if captura_ok:
            frame = padronizar_imagem(frame)
            exibir_video(frame)
            
except KeyboardInterrupt:
    captura_video.release()
    print("Interrompido")

Interrompido


Se executar o código agora você verá a si mesmo renderizado no Notebook.

O próximo passo é carregar o Dlib e os marcos faciais.

Vamos carregar o classificador com 68 pontos.

In [6]:
classificador_dlib_68_pontos = "classificadores/shape_predictor_68_face_landmarks.dat"
classificador_dlib = dlib.shape_predictor(classificador_dlib_68_pontos)
detector_face = dlib.get_frontal_face_detector()

In [7]:
def pontos_marcos_faciais(imagem):
    retangulos = detector_face(imagem, 1)

    if len(retangulos) == 0:
        return None

    marcos = []

    for retangulo in retangulos:
        marcos.append(np.matrix([[p.x, p.y] for p in classificador_dlib(imagem, retangulo).parts()]))

    return marcos

In [8]:
def anotar_marcos_faciais(imagem, marcos_faciais):
    if imagem is None:
        return im

    for marco in marcos_faciais:
        for idx, ponto in enumerate(marco):
            centro = (ponto[0, 0], ponto[0, 1])
            cv2.circle(imagem, centro, 3, color=(255, 255, 0), thickness=-1)
    return imagem

In [14]:
captura_video = cv2.VideoCapture(0)

try:
    while(True):
        captura_ok, frame = captura_video.read()

        if captura_ok:
            frame = padronizar_imagem(frame)
            pontos = pontos_marcos_faciais(frame)
            frame = anotar_marcos_faciais(frame, pontos)
            exibir_video(frame)
            
except KeyboardInterrupt:
    captura_video.release()
    print("Interrompido")

Interrompido


Vamos definir as constantes das regiões do rosto. Bastaria somente do lábio, mas é sempre bom colocar todas elas, caso sejam necessárias outras regiões.



In [15]:
FACE = list(range(17, 68))
FACE_COMPLETA = list(range(0, 68))
LABIO = list(range(48, 61))
SOMBRANCELHA_DIREITA = list(range(17, 22))
SOMBRANCELHA_ESQUERDA = list(range(22, 27))
OLHO_DIREITO = list(range(36, 42))
OLHO_ESQUERDO = list(range(42, 48))
NARIZ = list(range(27, 35))
MANDIBULA = list(range(0, 17))

In [16]:
def aspecto_razao_boca(pontos_boca):
    a = dist.euclidean(pontos_boca[3], pontos_boca[9])
    b = dist.euclidean(pontos_boca[2], pontos_boca[10])
    c = dist.euclidean(pontos_boca[4], pontos_boca[8])
    d = dist.euclidean(pontos_boca[0], pontos_boca[6])
    aspecto_razao = (a + b + c)/(3.0 * d)
    return aspecto_razao

Certo, agora vamos juntar tudo.

Arredondei para 3 casas decimais o valor de aspecto de razão, para não ficar muito grande na leitura em tela.

Note que peguei o primeiro ponto dos lábios e usei como referência para escrever o valor em tela.

Usei o método A1 do numpy para converter em array somente o primeiro par de pontos, depois converti para uma tupla para enfim usar como parâmetro x e y do putText. Assim o valor fica logo abaixo da boca, por isso que coloquei a margem de 20 pixels.

In [25]:
captura_video = cv2.VideoCapture(0)

try:
    while(True):
        captura_ok, frame = captura_video.read()

        if captura_ok:
            frame = padronizar_imagem(frame)
            pontos = pontos_marcos_faciais(frame)
            if pontos is not None:
                frame = anotar_marcos_faciais(frame, pontos)
                pontos = pontos[0]
                ar = aspecto_razao_boca(pontos[LABIO].tolist())
                ar = round(ar,3)
                coord = tuple(pontos[LABIO][0].A1.reshape(1, -1)[0])
                coord = (coord[0], coord[1] + 20)
                cv2.putText(frame, "Aspecto Razao " + str(ar), coord, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,0), 2)

            exibir_video(frame)

except KeyboardInterrupt:
    captura_video.release()
    print("Interrompido")

Interrompido


Faça um teste e veja o valor para sua boca de abertura que conseguiu obter. No meu caso, chegou aproximadamente em 0,5.

Com isso, se a medida de aspecto de razão for maior que 0,5, vamos desenhar um retângulo.

Para desenhar o retângulo tendo como base os pontos da boca, utilizamos o método boudingRect([pontos]).

Depois, com as coordenadas, desenhamos o retângulo, ficando desta forma:

In [26]:
captura_video = cv2.VideoCapture(0)

try:
    while(True):
        captura_ok, frame = captura_video.read()

        if captura_ok:
            frame = padronizar_imagem(frame)
            pontos = pontos_marcos_faciais(frame)
            if pontos is not None:
                frame = anotar_marcos_faciais(frame, pontos)
                pontos = pontos[0]
                ar = aspecto_razao_boca(pontos[LABIO].tolist())
                ar = round(ar,3)

                if ar > 0.5:
                    (x, y, w, h) = cv2.boundingRect(pontos[LABIO])
                    frame = cv2.rectangle(frame, (x,y), (x+w,y+h), (255,255,0), 2)

                coord = tuple(pontos[LABIO][0].A1.reshape(1, -1)[0])
                coord = (coord[0], coord[1] + 20)
                cv2.putText(frame, "Aspecto Razao " + str(ar), coord, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,0), 2)

            exibir_video(frame)

except KeyboardInterrupt:
    captura_video.release()
    print("Interrompido")

Interrompido


In [27]:
captura_video = cv2.VideoCapture(0)

try:
    while(True):
        captura_ok, frame = captura_video.read()

        if captura_ok:
            frame = padronizar_imagem(frame)
            pontos = pontos_marcos_faciais(frame)
            if pontos is not None:
                #frame = anotar_marcos_faciais(frame, pontos)
                pontos = pontos[0]
                ar = aspecto_razao_boca(pontos[LABIO].tolist())
                ar = round(ar,3)

                if ar > 0.5:
                    (x, y, w, h) = cv2.boundingRect(pontos[LABIO])
                    frame = cv2.rectangle(frame, (x,y), (x+w,y+h), (255,255,0), 2)

                coord = tuple(pontos[LABIO][0].A1.reshape(1, -1)[0])
                coord = (coord[0], coord[1] + 20)
                cv2.putText(frame, "Aspecto Razao " + str(ar), coord, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,0), 2)

            exibir_video(frame)

except KeyboardInterrupt:
    captura_video.release()
    print("Interrompido")

Interrompido


In [ ]:
def aspecto_razao_boca(pontos_boca):
    a = dist.euclidean(pontos_boca[3], pontos_boca[9])
    b = dist.euclidean(pontos_boca[2], pontos_boca[10])
    c = dist.euclidean(pontos_boca[4], pontos_boca[8])
    d = dist.euclidean(pontos_boca[0], pontos_boca[6])

    aspecto_razao = (a + b + c)/(3*d)
    